In [1]:
# #turn off gpu support, no need for it
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
#Import external functions
from image_processing import *
from utils import *
from custom_models import *
from inception_utils import *

from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, TensorBoard, EarlyStopping

import os

Using TensorFlow backend.


In [3]:
# Load images for training
path_to_images = '/home/pasok/Desktop/pmpakos2/data_generator/UCF-101-flow'

target_size = (240,320)
batch_size = 32


train_folder = path_to_images + '/train'
train_datagen = ImageDataGenerator(rotation_range=10,
                                   preprocessing_function=preprocess_input_flow,
                                   rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.33)
training_batches2 = train_datagen.flow_from_directory(train_folder,
                                                     target_size=target_size,
                                                     batch_size=batch_size,
                                                     interpolation='bicubic',
                                                     subset='training')
training_batches = crop_generator(training_batches2, crop_width, crop_height, random=True)

validation_batches2 = train_datagen.flow_from_directory(train_folder,
                                                       target_size=target_size,
                                                       batch_size=batch_size,
                                                       interpolation='bicubic',
                                                       subset='validation')
validation_batches = crop_generator(validation_batches2, crop_width, crop_height, random=True)

test_folder = path_to_images + '/test'
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_flow, rescale=1./255)
testing_batches2 = test_datagen.flow_from_directory(test_folder,
                                                   target_size=target_size,
                                                   batch_size=batch_size,
                                                   interpolation='bicubic')
testing_batches = crop_generator(testing_batches2, crop_width, crop_height, random=False)

classes_dictionary = {}
for cls, idx in training_batches2.class_indices.items():
    classes_dictionary[idx] = cls

Found 956573 images belonging to 101 classes.
Found 471071 images belonging to 101 classes.
Found 540949 images belonging to 101 classes.


In [4]:
# Single frame model definition
num_labels = 101
input_shape = (crop_height,crop_width,3)

#SGD Optimizer Parameters
base_lr = 0.0001
momentum = 0.9

In [5]:
model = get_model(weights='imagenet',input_shape=input_shape,num_labels=num_labels)
model = freeze_all_but_top(model)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 113, 113, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 113, 113, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
model.fit_generator(training_batches, 
                    steps_per_epoch=100,
                    validation_data=validation_batches,
                    validation_steps=10,
                    epochs=10,
                    callbacks=[])

Epoch 1/10
100/100 [==============================] - 39s 388ms/step - loss: 4.6894 - acc: 0.0328 - val_loss: 4.5887 - val_acc: 0.0156
Epoch 2/10
100/100 [==============================] - 34s 343ms/step - loss: 4.3634 - acc: 0.0572 - val_loss: 4.5335 - val_acc: 0.0250
Epoch 3/10
100/100 [==============================] - 34s 344ms/step - loss: 4.2027 - acc: 0.0859 - val_loss: 4.9471 - val_acc: 0.0312
Epoch 4/10
100/100 [==============================] - 34s 343ms/step - loss: 4.1299 - acc: 0.0853 - val_loss: 4.6275 - val_acc: 0.0156
Epoch 5/10
100/100 [==============================] - 34s 343ms/step - loss: 4.0306 - acc: 0.0966 - val_loss: 4.6789 - val_acc: 0.0312
Epoch 6/10
100/100 [==============================] - 35s 347ms/step - loss: 3.9542 - acc: 0.1047 - val_loss: 4.9278 - val_acc: 0.0250
Epoch 7/10
100/100 [==============================] - 34s 344ms/step - loss: 3.9193 - acc: 0.1159 - val_loss: 4.7562 - val_acc: 0.0250
Epoch 8/10
100/100 [==============================] - 3

In [7]:
filepath = 'saved_models/inception_flow.hdf5'
checkpointer = ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True)
# early_stopper = EarlyStopping(patience=10)
# tensorboard = TensorBoard(log_dir=os.path.join('data', 'logs-inception'))

model = freeze_all_but_mid_and_top(model=model,base_lr=base_lr,momentum=momentum)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 227, 227, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 113, 113, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 113, 113, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
model.fit_generator(training_batches,
                    steps_per_epoch=100,
                    validation_data=validation_batches,
                    validation_steps=10,
                    epochs=1000,
                    callbacks=[checkpointer])

Epoch 1/1000
100/100 [==============================] - 44s 440ms/step - loss: 3.7470 - acc: 0.1397 - top_k_categorical_accuracy: 0.3566 - val_loss: 4.3276 - val_acc: 0.0813 - val_top_k_categorical_accuracy: 0.2000

Epoch 00001: val_loss improved from inf to 4.32758, saving model to saved_models/inception_flow.hdf5
Epoch 2/1000
100/100 [==============================] - 37s 368ms/step - loss: 3.6517 - acc: 0.1556 - top_k_categorical_accuracy: 0.3822 - val_loss: 4.3236 - val_acc: 0.0625 - val_top_k_categorical_accuracy: 0.1781

Epoch 00002: val_loss improved from 4.32758 to 4.32363, saving model to saved_models/inception_flow.hdf5
Epoch 3/1000
100/100 [==============================] - 37s 367ms/step - loss: 3.6180 - acc: 0.1597 - top_k_categorical_accuracy: 0.3950 - val_loss: 4.3387 - val_acc: 0.0656 - val_top_k_categorical_accuracy: 0.1875

Epoch 00003: val_loss did not improve from 4.32363
Epoch 4/1000
100/100 [==============================] - 37s 367ms/step - loss: 3.5923 - acc: 0.

100/100 [==============================] - 36s 364ms/step - loss: 3.2079 - acc: 0.2206 - top_k_categorical_accuracy: 0.5078 - val_loss: 4.3463 - val_acc: 0.0531 - val_top_k_categorical_accuracy: 0.1781

Epoch 00030: val_loss did not improve from 4.14839
Epoch 31/1000
100/100 [==============================] - 36s 365ms/step - loss: 3.1579 - acc: 0.2269 - top_k_categorical_accuracy: 0.5184 - val_loss: 4.2935 - val_acc: 0.0750 - val_top_k_categorical_accuracy: 0.2094

Epoch 00031: val_loss did not improve from 4.14839
Epoch 32/1000
100/100 [==============================] - 36s 364ms/step - loss: 3.1611 - acc: 0.2369 - top_k_categorical_accuracy: 0.5125 - val_loss: 4.2934 - val_acc: 0.0656 - val_top_k_categorical_accuracy: 0.2219

Epoch 00032: val_loss did not improve from 4.14839
Epoch 33/1000
100/100 [==============================] - 36s 364ms/step - loss: 3.0851 - acc: 0.2453 - top_k_categorical_accuracy: 0.5266 - val_loss: 4.3590 - val_acc: 0.0563 - val_top_k_categorical_accuracy: 0

100/100 [==============================] - 36s 363ms/step - loss: 2.8200 - acc: 0.2963 - top_k_categorical_accuracy: 0.5891 - val_loss: 4.1922 - val_acc: 0.1000 - val_top_k_categorical_accuracy: 0.2531

Epoch 00090: val_loss did not improve from 3.99062
Epoch 91/1000
100/100 [==============================] - 36s 364ms/step - loss: 2.7660 - acc: 0.3059 - top_k_categorical_accuracy: 0.6088 - val_loss: 4.1108 - val_acc: 0.0781 - val_top_k_categorical_accuracy: 0.2906

Epoch 00091: val_loss did not improve from 3.99062
Epoch 92/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.7854 - acc: 0.3097 - top_k_categorical_accuracy: 0.6028 - val_loss: 3.9060 - val_acc: 0.1531 - val_top_k_categorical_accuracy: 0.3781

Epoch 00092: val_loss improved from 3.99062 to 3.90602, saving model to saved_models/inception_flow.hdf5
Epoch 93/1000
100/100 [==============================] - 36s 365ms/step - loss: 2.7570 - acc: 0.3144 - top_k_categorical_accuracy: 0.6063 - val_loss: 4.1926


Epoch 00150: val_loss did not improve from 3.90602
Epoch 151/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.5338 - acc: 0.3537 - top_k_categorical_accuracy: 0.6578 - val_loss: 4.1308 - val_acc: 0.1031 - val_top_k_categorical_accuracy: 0.2750

Epoch 00151: val_loss did not improve from 3.90602
Epoch 152/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.5219 - acc: 0.3659 - top_k_categorical_accuracy: 0.6653 - val_loss: 4.1435 - val_acc: 0.1062 - val_top_k_categorical_accuracy: 0.3187

Epoch 00152: val_loss did not improve from 3.90602
Epoch 153/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.4621 - acc: 0.3672 - top_k_categorical_accuracy: 0.6759 - val_loss: 4.2374 - val_acc: 0.0813 - val_top_k_categorical_accuracy: 0.3000

Epoch 00153: val_loss did not improve from 3.90602
Epoch 154/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.4950 - acc: 0.3709 - top_k_categorical_accuracy: 0.6578 - 

100/100 [==============================] - 36s 363ms/step - loss: 2.3090 - acc: 0.3994 - top_k_categorical_accuracy: 0.7000 - val_loss: 4.1836 - val_acc: 0.1344 - val_top_k_categorical_accuracy: 0.3125

Epoch 00211: val_loss did not improve from 3.89016
Epoch 212/1000
100/100 [==============================] - 36s 364ms/step - loss: 2.3909 - acc: 0.3972 - top_k_categorical_accuracy: 0.6906 - val_loss: 4.0297 - val_acc: 0.1281 - val_top_k_categorical_accuracy: 0.3187

Epoch 00212: val_loss did not improve from 3.89016
Epoch 213/1000
100/100 [==============================] - 36s 363ms/step - loss: 2.3520 - acc: 0.3997 - top_k_categorical_accuracy: 0.6994 - val_loss: 4.1497 - val_acc: 0.1094 - val_top_k_categorical_accuracy: 0.2687

Epoch 00213: val_loss did not improve from 3.89016
Epoch 214/1000
100/100 [==============================] - 36s 364ms/step - loss: 2.2971 - acc: 0.4172 - top_k_categorical_accuracy: 0.7006 - val_loss: 4.2880 - val_acc: 0.0938 - val_top_k_categorical_accuracy

...


Epoch 00635: val_loss did not improve from 3.73971
Epoch 636/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.6503 - acc: 0.5694 - top_k_categorical_accuracy: 0.8175 - val_loss: 4.0349 - val_acc: 0.1187 - val_top_k_categorical_accuracy: 0.3406

Epoch 00636: val_loss did not improve from 3.73971
Epoch 637/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.6988 - acc: 0.5544 - top_k_categorical_accuracy: 0.8156 - val_loss: 3.9839 - val_acc: 0.1250 - val_top_k_categorical_accuracy: 0.2938

Epoch 00637: val_loss did not improve from 3.73971
Epoch 638/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.6787 - acc: 0.5528 - top_k_categorical_accuracy: 0.8103 - val_loss: 3.9899 - val_acc: 0.1375 - val_top_k_categorical_accuracy: 0.3406

Epoch 00638: val_loss did not improve from 3.73971
Epoch 639/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.6288 - acc: 0.5775 - top_k_categorical_accuracy: 0.8241 - 


Epoch 00696: val_loss did not improve from 3.73971
Epoch 697/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.5981 - acc: 0.5766 - top_k_categorical_accuracy: 0.8213 - val_loss: 3.9436 - val_acc: 0.1156 - val_top_k_categorical_accuracy: 0.3375

Epoch 00697: val_loss did not improve from 3.73971
Epoch 698/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.5820 - acc: 0.5841 - top_k_categorical_accuracy: 0.8331 - val_loss: 4.0656 - val_acc: 0.1281 - val_top_k_categorical_accuracy: 0.3094

Epoch 00698: val_loss did not improve from 3.73971
Epoch 699/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.6055 - acc: 0.5753 - top_k_categorical_accuracy: 0.8291 - val_loss: 3.9301 - val_acc: 0.1281 - val_top_k_categorical_accuracy: 0.3438

Epoch 00699: val_loss did not improve from 3.73971
Epoch 700/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.6037 - acc: 0.5831 - top_k_categorical_accuracy: 0.8253 - 


Epoch 00757: val_loss did not improve from 3.73971
Epoch 758/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.5020 - acc: 0.6075 - top_k_categorical_accuracy: 0.8481 - val_loss: 4.0562 - val_acc: 0.1531 - val_top_k_categorical_accuracy: 0.3406

Epoch 00758: val_loss did not improve from 3.73971
Epoch 759/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.5888 - acc: 0.5828 - top_k_categorical_accuracy: 0.8294 - val_loss: 3.9667 - val_acc: 0.1281 - val_top_k_categorical_accuracy: 0.3375

Epoch 00759: val_loss did not improve from 3.73971
Epoch 760/1000
100/100 [==============================] - 36s 365ms/step - loss: 1.5009 - acc: 0.5981 - top_k_categorical_accuracy: 0.8441 - val_loss: 4.0133 - val_acc: 0.1469 - val_top_k_categorical_accuracy: 0.3250

Epoch 00760: val_loss did not improve from 3.73971
Epoch 761/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.5820 - acc: 0.5825 - top_k_categorical_accuracy: 0.8275 - 


Epoch 00818: val_loss did not improve from 3.73971
Epoch 819/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.4826 - acc: 0.6047 - top_k_categorical_accuracy: 0.8469 - val_loss: 4.1388 - val_acc: 0.1094 - val_top_k_categorical_accuracy: 0.3125

Epoch 00819: val_loss did not improve from 3.73971
Epoch 820/1000
100/100 [==============================] - 36s 363ms/step - loss: 1.4907 - acc: 0.6100 - top_k_categorical_accuracy: 0.8394 - val_loss: 3.9371 - val_acc: 0.1437 - val_top_k_categorical_accuracy: 0.3531

Epoch 00820: val_loss did not improve from 3.73971
Epoch 821/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.5156 - acc: 0.6056 - top_k_categorical_accuracy: 0.8409 - val_loss: 3.9041 - val_acc: 0.1344 - val_top_k_categorical_accuracy: 0.3594

Epoch 00821: val_loss did not improve from 3.73971
Epoch 822/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.5112 - acc: 0.5913 - top_k_categorical_accuracy: 0.8428 - 


Epoch 00879: val_loss did not improve from 3.73971
Epoch 880/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.3995 - acc: 0.6341 - top_k_categorical_accuracy: 0.8594 - val_loss: 3.9494 - val_acc: 0.1375 - val_top_k_categorical_accuracy: 0.3406

Epoch 00880: val_loss did not improve from 3.73971
Epoch 881/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.4123 - acc: 0.6291 - top_k_categorical_accuracy: 0.8563 - val_loss: 4.1421 - val_acc: 0.1313 - val_top_k_categorical_accuracy: 0.3250

Epoch 00881: val_loss did not improve from 3.73971
Epoch 882/1000
100/100 [==============================] - 36s 364ms/step - loss: 1.4517 - acc: 0.6175 - top_k_categorical_accuracy: 0.8472 - val_loss: 3.9824 - val_acc: 0.1281 - val_top_k_categorical_accuracy: 0.3312

Epoch 00882: val_loss did not improve from 3.73971
Epoch 883/1000
100/100 [==============================] - 36s 365ms/step - loss: 1.4368 - acc: 0.6125 - top_k_categorical_accuracy: 0.8534 - 

In [ ]:
scores = model.evaluate_generator(generator=testing_batches,verbose=1,steps=testing_batches2.samples//batch_size)
print(scores)
